## Instalar Bibliotecas

In [ ]:
# Instalar Bibliotecas
import pandas as pd
import numpy as np
import pandas as pd
import io
import torch

## Monta Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Preparar OpenNMT

In [ ]:
def baixarMT():
  !mkdir -p nmt
  %cd nmt
  !git clone https://github.com/ymoslem/MT-Preparation.git

def prepararBleuScript():
  # Download the BLEU script
  !wget https://raw.githubusercontent.com/ymoslem/MT-Evaluation/main/BLEU/compute-bleu.py

  # Install sacrebleu
  !pip3 install sacrebleu

In [ ]:
#baixarMT()
!pip3 install -r /content/drive/MyDrive/tcc/nmt/MT-Preparation/requirements.txt
%cd "/content/drive/MyDrive/tcc/nmt"

/content/drive/MyDrive/tcc/nmt


In [ ]:
# Instalar Open NMT
!pip3 install OpenNMT-py

# If needed install/update sentencepiece
!pip3 install --upgrade -q sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 257.3/257.3 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 36.8/36.8 MB 47.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.7/56.7 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.0/17.0 MB 68.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.3/106.3 kB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 106.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.8/110.8 kB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 47.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.7/227.7 kB 23.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 19.0 MB/s eta 0:00:00


## Gerar Dados

In [ ]:
def lerDadosTSAR():
  sentence_list = []
  mask_word_list = []
  first_label_list = []

  arquivo =  '/content/drive/MyDrive/tcc/BERT-LS-master/datasets/tsar2022_pt_test_gold.tsv'
  with open(arquivo, "r", encoding='UTF-8') as reader:
    while True:
        line = reader.readline()
        if not line:
          break

        sentence,words = line.strip().split('\t',1)
        mask_word,labels = words.strip().split('\t',1)
        label = labels.split('\t')
        first_label = label[0]

        sentence_list.append(sentence)
        mask_word_list.append(mask_word)
        first_label_list.append(first_label)


  data_final = pd.DataFrame({'sentence': sentence_list, 'mask_word': mask_word_list, 'first_label': first_label_list})
  return data_final

def lerDadosSimplex():
  import pandas as pd
  arquivo = '/content/drive/MyDrive/tcc/BERT-LS-master/datasets/Simplex 3.0.xlsx'
  data = pd.pandas.read_excel(arquivo)

  df = data[["sentença", "palavra_dificil", "sinônimos_ranqueados"]]
  #df = df.iloc[942:]
  df = df.drop(941)
  df = df.dropna()
  dict_resultado = []

  for row in df.values:
    resultado = []
    sentenca = row[0]
    palavra_complexa = row[1]
    sinonimos_ranqueados = row[2]

    resultado.append(sentenca.replace("*","").replace("'","\'").replace("´","\´"))
    resultado.append(palavra_complexa)

    sinonimos = sinonimos_ranqueados.replace('[', '').replace(']', '').replace("'", '')
    list_sinonimos = sinonimos.split(',')
    for sinonimo in list_sinonimos:
      if sinonimo.strip() != palavra_complexa.strip():
        resultado.append(sinonimo)

    dict_resultado.append(resultado)

  data_final = pd.DataFrame(dict_resultado)
  data_final.fillna(value='', inplace=True)
  data_final = data_final.iloc[:, 0:3]

  data_final.columns =['sentence', 'mask_word', 'first_label']
  return data_final

def gerarArquivosMT(data, from_file, to_file):
  data.replace(np.nan, '', inplace=True)
  t = {'original': [], 'simplificado': []}

  for indice_linha, linha in data.iterrows():
    total_colunas = range(len(linha))
    #for indice_coluna in total_colunas:
      #if (indice_coluna > 1):
    frase_original =  linha[0]
    #frase_original = frase_original.replace('"', ' ')
    palavra_complexa = linha[1]
    palavra_substituta = linha[2]

    if (palavra_substituta != ''):
      frase_nova = frase_original.replace(palavra_complexa, palavra_substituta)

      t['original'].append(frase_original)
      t['simplificado'].append(frase_nova)


  df = pd.DataFrame(t)
  df_distinct = df.drop_duplicates()

  path = "/content/drive/MyDrive/tcc/nmt/"
  df['original'].to_csv(path + from_file, index=False, header=False)
  df['simplificado'].to_csv(path + to_file, index=False, header=False)

In [ ]:
data = lerDadosTSAR()
from_file = "tsar_from"
to_file = "tsar_to"

gerarArquivosMT(data, from_file, to_file)

In [ ]:
data2 = lerDadosSimplex()
from_file = "simplex_from"
to_file = "simplex_to"

gerarArquivosMT(data2, from_file, to_file)

In [ ]:
data3 = pd.concat([data, data2], ignore_index=True)

from_file = "tsar_simplex_from"
to_file = "tsar_simplex_to"
gerarArquivosMT(data3, from_file, to_file)

## Funções

In [ ]:
def configurarYAML(source_file_filtered_subword_train, target_file_filtered_subword_train, source_file_filtered_subword_dev, target_file_filtered_subword_dev):
  config = '''# config.yaml


  ## Where the samples will be written
  save_data: run

  # Training files
  data:
      corpus_1:
          path_src: ''' + source_file_filtered_subword_train + '''
          path_tgt: ''' + target_file_filtered_subword_train + '''
          transforms: [filtertoolong]
      valid:
          path_src: ''' + source_file_filtered_subword_dev + '''
          path_tgt: ''' + target_file_filtered_subword_dev + '''
          transforms: [filtertoolong]

  # Vocabulary files, generated by onmt_build_vocab
  src_vocab: run/source.vocab
  tgt_vocab: run/target.vocab

  # Vocabulary size - should be the same as in sentence piece
  src_vocab_size: 10000
  tgt_vocab_size: 10000

  # Filter out source/target longer than n if [filtertoolong] enabled
  src_seq_length: 350
  src_seq_length: 350

  # Tokenization options
  src_subword_model: source.model
  tgt_subword_model: target.model

  # Where to save the log file and the output models/checkpoints
  log_file: train.log
  save_model: models/''' + model_name + '''

  # Stop training if it does not imporve after n validations
  early_stopping: 4

  # Default: 5000 - Save a model checkpoint for each n
  save_checkpoint_steps: 1000

  # To save space, limit checkpoints to last n
  # keep_checkpoint: 3

  seed: 3435

  # Default: 100000 - Train the model to max n steps
  # Increase to 200000 or more for large datasets
  # For fine-tuning, add up the required steps to the original steps
  train_steps: 100000

  # Default: 10000 - Run validation after n steps
  valid_steps: 5000

  # Default: 4000 - for large datasets, try up to 8000
  warmup_steps: 4000
  report_every: 100

  # Number of GPUs, and IDs of GPUs
  world_size: 1
  gpu_ranks: [0]

  # Batching
  bucket_size: 262144
  num_workers: 2  # Default: 2, set to 0 when RAM out of memory
  batch_type: "tokens"
  batch_size: 4096   # Tokens per batch, change when CUDA out of memory
  valid_batch_size: 2048
  max_generator_batches: 2
  accum_count: [4]
  accum_steps: [0]

  # Optimization
  model_dtype: "fp16"
  optim: "adam"
  learning_rate: 2
  # warmup_steps: 8000
  decay_method: "noam"
  adam_beta2: 0.998
  max_grad_norm: 0
  label_smoothing: 0.1
  param_init: 0
  param_init_glorot: true
  normalization: "tokens"

  # Model
  encoder_type: transformer
  decoder_type: transformer
  position_encoding: true
  enc_layers: 6
  dec_layers: 6
  heads: 8
  hidden_size: 512
  word_vec_size: 512
  transformer_ff: 2048
  dropout_steps: [0]
  dropout: [0.1]
  attention_dropout: [0.1]
  '''

  with open("config.yaml", "w+") as config_yaml:
    config_yaml.write(config)

  !cat config.yaml

def construirVocabulario():
  # Build Vocabulary

  # -config: path to your config.yaml file
  # -n_sample: use -1 to build vocabulary on all the segment in the training dataset
  # -num_threads: change it to match the number of CPUs to run it faster

  %cd "/content/drive/MyDrive/tcc/nmt"
  !onmt_build_vocab -config config.yaml -n_sample -1 -num_threads 12

def traduzir(model_name, source_file_filtered_subword_test, file_translated):
  %cd "/content/drive/MyDrive/tcc/nmt"

  model_name_step_500 = "models/" + model_name + "_step_500.pt"
  model_name_step_1000 = "models/" + model_name + "_step_1000.pt"
  model_name_step_2000 = "models/" + model_name + "_step_2000.pt"
  model_name_step_3000 = "models/" + model_name + "_step_3000.pt"

  # Translate the "subworded" source file of the test dataset
  # Change the model name, if needed.
  !onmt_translate -model $model_name_step_1000 -src $source_file_filtered_subword_test -output $file_translated -gpu 0 -min_length 1

def avaliar(target_file_filtered_subword_test_desubword, file_translated_desubword):
  prepararBleuScript()

  # Evaluate the translation (without subwording)
  !python3 compute-bleu.py $target_file_filtered_subword_test_desubword $file_translated_desubword

## Configurações Iniciais

In [ ]:
# Definir Variaveis
# Filter the dataset
# Arguments: source file, target file, source language, target language

name_dataset =  "tsar_simplex"

source_file = name_dataset + "_from"
target_file = name_dataset + "_to"

source_language = "pt_BR"
target_language = "pt_BR"

source_file_filtered = source_file + "_filtered"
target_file_filtered = target_file + "_filtered"

# Nomes dos arquivos filtrados e com subword
source_file_filtered_subword = source_file + "_filtered_subword"
target_file_filtered_subword = target_file + "_filtered_subword"

# Nomes dos arquivos filtrados e com subword dividos em dev, test e train
source_file_filtered_subword_dev   = source_file + "_filtered_subword_dev"
source_file_filtered_subword_test  = source_file + "_filtered_subword_test"
source_file_filtered_subword_train = source_file + "_filtered_subword_train"
target_file_filtered_subword_dev   = target_file + "_filtered_subword_dev"
target_file_filtered_subword_test  = target_file + "_filtered_subword_test"
target_file_filtered_subword_train = target_file + "_filtered_subword_train"

# Nomes dos arquivos filtrados e com subword
source_file_filtered_subword_dev_desubword   = source_file + "_filtered_subword_dev_desubword"
source_file_filtered_subword_test_desubword  = source_file + "_filtered_subword_test_desubword"
source_file_filtered_subword_train_desubword = source_file + "_filtered_subword_train_desubword"
target_file_filtered_subword_dev_desubword   = target_file + "_filtered_subword_dev_desubword"
target_file_filtered_subword_test_desubword  = target_file + "_filtered_subword_test_desubword"
target_file_filtered_subword_train_desubword = target_file + "_filtered_subword_train_desubword"

# Arquivos traduzidos
file_translated = name_dataset + "_translated"
file_translated_desubword = name_dataset + "_translated_desubword"


model_name = "model_simplificador_" + name_dataset

## Principal

### Pré-processamento

In [ ]:
# Filter
!python3 MT-Preparation/filtering/filter.py $source_file $target_file $source_language $target_language

# Train a SentencePiece model for subword tokenization
!python3 MT-Preparation/subwording/1-train_bpe.py $source_file_filtered $target_file_filtered

# Subword the dataset
!python3 MT-Preparation/subwording/2-subword.py source.model target.model $source_file_filtered $target_file_filtered

# Split the dataset into training set, development set, and test set
# Development and test sets should be between 1000 and 5000 segments (here we chose 2000)
!python3 MT-Preparation/train_dev_split/train_dev_test_split.py $source_file_filtered_subword $target_file_filtered_subword


Dataframe shape (rows, columns): (979, 2)
--- Rows with Empty Cells Deleted	--> Rows: 979
--- Duplicates Deleted			--> Rows: 974
--- Source-Copied Rows Deleted		--> Rows: 974
--- Too Long Source/Target Deleted	--> Rows: 974
--- HTML Removed			--> Rows: 974
--- Rows will remain in true-cased	--> Rows: 974
--- Rows with Empty Cells Deleted	--> Rows: 974
--- Rows Shuffled			--> Rows: 974
--- Source Saved: tsar_simplex_from_filtered
--- Target Saved: tsar_simplex_to_filtered
sentencepiece_trainer.cc(178) LOG(INFO) Running command: --input=tsar_simplex_from_filtered --model_prefix=source --vocab_size=10000 --hard_vocab_limit=false --model_type=bpe --split_digits=true
sentencepiece_trainer.cc(78) LOG(INFO) Starts training with : 
trainer_spec {
  input: tsar_simplex_from_filtered
  input_format: 
  model_prefix: source
  model_type: BPE
  vocab_size: 10000
  self_test_sample_size: 0
  character_coverage: 0.9995
  input_sentence_size: 0
  shuffle_input_sentence: 1
  seed_sentencepiece_size: 1

### Conteúdo do Arquivos

In [ ]:
# First 3 lines before filter
!head -n 3 $source_file && echo "-----" && head -n 3 $target_file && echo "-----"

# First 3 lines before subwording
!head -n 3 $source_file_filtered && echo "-----" && head -n 3 $target_file_filtered

# First 3 lines after subwording
!head -n 3 $source_file_filtered_subword && echo "---" && head -n 3 $target_file_filtered_subword

# Line count for the subworded train, dev, test datatest
!wc -l *_subword_*

# Check the first and last line from each dataset
!echo "---First line---"
!head -n 1 *_{train,dev,test}

!echo -e "\n---Last line---"
!tail -n 1 *_{train,dev,test}

ao prever a distribuição gratuita de uniforme para alunos da rede pública um projeto em discussão no congresso institui uma controvérsia em escolas prefeituras e secretarias de educação
a matéria ainda precisa passar pela câmara dos deputados e pela presidência da república antes de entrar em vigor
o acessório polêmico entrou no projeto de autoria do senador cícero lucena psdbpb graças a uma emenda aprovada na comissão de educação do senado em outubro
-----
ao prever a distribuição gratuita de uniforme para alunos da rede pública um projeto em discussão no congresso institui uma polêmica em escolas prefeituras e secretarias de educação
a assunto ainda precisa passar pela câmara dos deputados e pela presidência da república antes de entrar em vigor
o acessório controverso entrou no projeto de autoria do senador cícero lucena psdbpb graças a uma emenda aprovada na comissão de educação do senado em outubro
-----
`` Sua estratégia é perfeita .
a realização é fruto de uma parceria entre a c

### Configurar YAML

In [ ]:
configurarYAML(source_file_filtered_subword_train, target_file_filtered_subword_train, source_file_filtered_subword_dev, target_file_filtered_subword_dev)

# config.yaml


  ## Where the samples will be written
  save_data: run

  # Training files
  data:
      corpus_1:
          path_src: tsar_simplex_from_filtered_subword_train
          path_tgt: tsar_simplex_to_filtered_subword_train
          transforms: [filtertoolong]
      valid:
          path_src: tsar_simplex_from_filtered_subword_dev
          path_tgt: tsar_simplex_to_filtered_subword_dev
          transforms: [filtertoolong]

  # Vocabulary files, generated by onmt_build_vocab
  src_vocab: run/source.vocab
  tgt_vocab: run/target.vocab

  # Vocabulary size - should be the same as in sentence piece
  src_vocab_size: 10000
  tgt_vocab_size: 10000

  # Filter out source/target longer than n if [filtertoolong] enabled
  src_seq_length: 350
  src_seq_length: 350

  # Tokenization options
  src_subword_model: source.model
  tgt_subword_model: target.model

  # Where to save the log file and the output models/checkpoints
  log_file: train.log
  save_model: models/model_simplificad

### Construir vocabulário

In [ ]:
construirVocabulario()

/content/drive/MyDrive/tcc/nmt
Corpus corpus_1's weight should be given. We default it to 1 for you.
[2024-02-19 17:09:40,392 INFO] Counter vocab from -1 samples.
[2024-02-19 17:09:40,392 INFO] n_sample=-1: Build vocab on full datasets.
[2024-02-19 17:09:40,544 INFO] Counters src: 5691
[2024-02-19 17:09:40,544 INFO] Counters tgt: 5643
Traceback (most recent call last):
  File "/usr/local/bin/onmt_build_vocab", line 8, in <module>
    sys.exit(main())
  File "/usr/local/lib/python3.10/dist-packages/onmt/bin/build_vocab.py", line 283, in main
    build_vocab_main(opts)
  File "/usr/local/lib/python3.10/dist-packages/onmt/bin/build_vocab.py", line 267, in build_vocab_main
    save_counter(src_counter, opts.src_vocab)
  File "/usr/local/lib/python3.10/dist-packages/onmt/bin/build_vocab.py", line 256, in save_counter
    check_path(save_path, exist_ok=opts.overwrite, log=logger.warning)
  File "/usr/local/lib/python3.10/dist-packages/onmt/utils/misc.py", line 47, in check_path
    raise IOE

### Configurações máquinas

In [ ]:
# Find the number of CPUs/cores on the machine
!nproc --all

# Check if the GPU is active
!nvidia-smi -L

# Check if the GPU is visable to PyTorch
print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))

gpu_memory = torch.cuda.mem_get_info(0)
print("Free GPU memory:", gpu_memory[0]/1024**2, "out of:", gpu_memory[1]/1024**2)

# For error debugging try:
# !dmesg -T

8
GPU 0: Tesla V100-SXM2-16GB (UUID: GPU-161437e6-f656-80b2-0a57-737259a8c3ce)
True
Tesla V100-SXM2-16GB
Free GPU memory: 15842.375 out of: 16151.125


## Treinar Modelo

In [ ]:
# Train the NMT model
#!onmt_train -config config.yaml

## Traduzir

In [ ]:
traduzir(model_name, source_file_filtered_subword_test, file_translated)

/content/drive/MyDrive/tcc/nmt
[2024-02-19 17:09:46,013 INFO] Loading checkpoint from models/model_simplificador_tsar_simplex_step_1000.pt
[2024-02-19 17:09:50,423 INFO] Loading data into the model
[2024-02-19 17:09:53,376 INFO] PRED SCORE: -0.0653, PRED PPL: 1.07 NB SENTENCES: 97
Time w/o python interpreter load/terminate:  7.828631639480591


## Conteúdo Traduzido

In [ ]:
!head -n 10 $file_translated

▁" Atualmente ▁, ▁para ▁ter ▁acesso ▁aos ▁jogos ▁mais ▁recentes ▁, ▁os ▁brasileiros ▁têm ▁de ▁usar ▁aos ▁importados ▁, ▁que ▁chegam ▁ao ▁Brasil ▁custando ▁até ▁o ▁qu ádruplo ▁do ▁preço ▁."
▁" Já ▁outro ▁gênero ▁, ▁cujas ▁espécies ▁geral mente ▁possuem ▁cores ▁metálicas ▁radia ntes ▁, ▁foi ▁batizado ▁de ▁."
▁" Na ▁triste ▁cidade ▁de ▁Mar - Nublado ▁, ▁a ▁menina ▁Tilda ▁já ▁se ▁acostumou ▁a ▁ver ▁a ▁plaquinha ▁`` ▁Vende - se ” ▁, ▁na ▁casa ▁ao ▁lado ▁."
▁o ▁anúncio ▁foi ▁feito ▁pelo ▁bispo ▁durante ▁uma ▁audiência ▁geral ▁na ▁qual ▁o ▁papa ▁informou ▁que ▁vai ▁realizar ▁uma ▁nova ▁assembléia ▁de ▁cardeais ▁chamada ▁de ▁cons istório
▁" Há ▁mais ▁de ▁ 6 0 0 ▁espécies ▁de ▁arraias ▁no ▁mundo ▁, ▁de ▁água ▁doce ▁e ▁salgada ▁, ▁sendo ▁que ▁cerca ▁de ▁ 7 0 ▁existem ▁só ▁no ▁Brasil ▁."
▁"`` ▁É ▁complicado ▁dizer ▁qual ▁é ▁a ▁melhor ▁e ▁até ▁o ▁que ▁é ▁uma ▁boa ▁escola ▁, ▁pois ▁isso ▁reflete ▁o ▁que ▁a ▁sociedade ▁pede ▁e ▁muda ▁com ▁o ▁passar ▁do ▁tempo ▁'' ▁, ▁explica ▁Alfredo ▁Matos ▁Moura ▁

In [ ]:
!head -n 10 $source_file_filtered_subword_test

▁" Atualmente ▁, ▁para ▁ter ▁acesso ▁aos ▁jogos ▁mais ▁recentes ▁, ▁os ▁brasileiros ▁têm ▁de ▁recorrer ▁aos ▁importados ▁, ▁que ▁chegam ▁ao ▁Brasil ▁custando ▁até ▁o ▁quádrupl o ▁do ▁preço ▁."
▁" Já ▁outro ▁gênero ▁, ▁cujas ▁espécies ▁geral mente ▁possuem ▁cores ▁metálicas ▁reluz entes ▁, ▁foi ▁batizado ▁de ▁."
▁" Na ▁melanc ólica ▁cidade ▁de ▁Mar - Nublado ▁, ▁a ▁menina ▁Tilda ▁já ▁se ▁acostumou ▁a ▁ver ▁a ▁pl aquinha ▁`` ▁Vende - se ” ▁, ▁na ▁casa ▁ao ▁lado ▁."
▁o ▁anúncio ▁foi ▁feito ▁pelo ▁pontífice ▁durante ▁uma ▁audi ência ▁geral ▁na ▁qual ▁o ▁papa ▁informou ▁que ▁vai ▁realizar ▁uma ▁nova ▁assembléia ▁de ▁cardeais ▁chamada ▁de ▁consistó rio
▁" Há ▁mais ▁de ▁ 6 0 0 ▁espécies ▁de ▁raias ▁no ▁mundo ▁, ▁de ▁água ▁doce ▁e ▁salgada ▁, ▁sendo ▁que ▁cerca ▁de ▁ 7 0 ▁existem ▁só ▁no ▁Brasil ▁."
▁"`` ▁É ▁complicado ▁dizer ▁qual ▁é ▁a ▁melhor ▁e ▁até ▁o ▁que ▁é ▁uma ▁boa ▁escola ▁, ▁pois ▁isso ▁reflete ▁o ▁que ▁a ▁sociedade ▁demanda ▁e ▁muda ▁com ▁o ▁passar ▁do ▁tempo ▁'' ▁, ▁explica ▁Alfre

In [ ]:
!head -n 10 $target_file_filtered_subword_test_desubword

a pesquisa que sai publicada hoje na revista britânica nature wwwnaturecom é a primeira a perceber que um buraco negro está orbitando o centro da via láctea fora do plano do disco galáctico
"Os adultos ficaram de fora , mas asseguro que vão adorar assistir !"
"No entanto , o mar profundo é o lar de algumas espécies marinhas e palco da pesquisa da viagem francesa Bicose , que ancorou em fevereiro entre as Ilhas Canárias e Guadalupe , no Oceano Atlântico ."
A Terra demora 24 horas para completar um rotação em volta de si mesma .
"Eles concluíram , por exemplo , que tinha uma audição apurada , que poderia ser utilizada na comunicação com outros membros da mesma espécie ou na reconhecença de presas ."
"O problema é que as regiões tropicais têm sido as mais atingidas pelo aquecimento mundial : o aumento da temperatura do planeta , registrado nas últimas décadas ."
veste terno azul e gravata corderosa perfeitos mas chega com um andar desengonçado
E para você que já quer correr para a janela 

In [ ]:
# Check the first 10 lines of the translation file
!head -n 10 $file_translated

# Desubword the translation file
!python3 MT-Preparation/subwording/3-desubword.py target.model $file_translated

# Check the first 10 lines of the desubworded translation file
!head -n 10 $file_translated_desubword

# Desubword the target file (reference) of the test dataset
# Note: You might as well have split files *before* subwording during dataset preperation,
# but sometimes datasets have tokeniztion issues, so this way you are sure the file is really untokenized.
!python3 MT-Preparation/subwording/3-desubword.py target.model $target_file_filtered_subword_test

# Check the first 5 lines of the desubworded reference
!head -n 10 $target_file_filtered_subword_test_desubword

▁" Atualmente ▁, ▁para ▁ter ▁acesso ▁aos ▁jogos ▁mais ▁recentes ▁, ▁os ▁brasileiros ▁têm ▁de ▁usar ▁aos ▁importados ▁, ▁que ▁chegam ▁ao ▁Brasil ▁custando ▁até ▁o ▁qu ádruplo ▁do ▁preço ▁."
▁" Já ▁outro ▁gênero ▁, ▁cujas ▁espécies ▁geral mente ▁possuem ▁cores ▁metálicas ▁radia ntes ▁, ▁foi ▁batizado ▁de ▁."
▁" Na ▁triste ▁cidade ▁de ▁Mar - Nublado ▁, ▁a ▁menina ▁Tilda ▁já ▁se ▁acostumou ▁a ▁ver ▁a ▁plaquinha ▁`` ▁Vende - se ” ▁, ▁na ▁casa ▁ao ▁lado ▁."
▁o ▁anúncio ▁foi ▁feito ▁pelo ▁bispo ▁durante ▁uma ▁audiência ▁geral ▁na ▁qual ▁o ▁papa ▁informou ▁que ▁vai ▁realizar ▁uma ▁nova ▁assembléia ▁de ▁cardeais ▁chamada ▁de ▁cons istório
▁" Há ▁mais ▁de ▁ 6 0 0 ▁espécies ▁de ▁arraias ▁no ▁mundo ▁, ▁de ▁água ▁doce ▁e ▁salgada ▁, ▁sendo ▁que ▁cerca ▁de ▁ 7 0 ▁existem ▁só ▁no ▁Brasil ▁."
▁"`` ▁É ▁complicado ▁dizer ▁qual ▁é ▁a ▁melhor ▁e ▁até ▁o ▁que ▁é ▁uma ▁boa ▁escola ▁, ▁pois ▁isso ▁reflete ▁o ▁que ▁a ▁sociedade ▁pede ▁e ▁muda ▁com ▁o ▁passar ▁do ▁tempo ▁'' ▁, ▁explica ▁Alfredo ▁Matos ▁Moura ▁

## Avaliar

In [ ]:
avaliar(target_file_filtered_subword_test_desubword, file_translated_desubword)

--2024-02-19 17:09:56--  https://raw.githubusercontent.com/ymoslem/MT-Evaluation/main/BLEU/compute-bleu.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 957 [text/plain]
Saving to: ‘compute-bleu.py.1’

compute-bleu.py.1   100%[===================>]     957  --.-KB/s    in 0s      

2024-02-19 17:09:56 (38.2 MB/s) - ‘compute-bleu.py.1’ saved [957/957]

Reference 1st sentence: "Atualmente , para ter acesso aos jogos mais recentes , os brasileiros têm de usar aos importados , que chegam ao Brasil custando até o quádruplo do preço ."
MTed 1st sentence: "Atualmente , para ter acesso aos jogos mais recentes , os brasileiros têm de usar aos importados , que chegam ao Brasil custando até o quádruplo do preço ."
BLEU:  89.11461413491396


## Remover Modelo

In [ ]:
#!rm -rf /nmt/models/